In [59]:
# IMPORTING NECESSARY LIBRARIES

In [60]:
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model

from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Sequential


In [61]:
# IMAGE RESIZING

In [62]:
IMAGE_SIZE = [224,224]
train_path = 'Train/'
test_path = 'Test/'

In [63]:
# IMPORTING RESNET50 AND USING IMAGENET WEIGHTS

In [64]:
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights="imagenet", include_top=False)

In [ ]:
resnet.summary()

In [66]:
# NOT TRAINING EXISTING WEIGHTS

In [67]:
for layer in resnet.layers:
    layer.trainable = False

In [68]:
folders = glob('Train/*')

In [ ]:
folders

In [70]:
x = Flatten()(resnet.output)

In [71]:
prediction = Dense(len(folders), activation='softmax')(x)

In [72]:
base_model = Model(inputs=resnet.input, outputs = prediction)

In [ ]:
base_model.summary()

In [74]:
base_model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [75]:
# DATA AUGUMENTATION

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale =1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('Train',
                                                 target_size=(224,224),
                                                 batch_size=32,
                                                 class_mode = 'categorical')

In [ ]:
test_set = test_datagen.flow_from_directory('Test',
                                            target_size=(224,224),
                                            batch_size=32,
                                            class_mode = 'categorical')

In [ ]:
model1 = base_model.fit(
    training_set,
    validation_data = test_set,
    epochs=50
)

In [ ]:
model1.history

In [ ]:
# PLOT ACCURACY AND LOSS
plt.plot(model1.history['loss'], label=['train_loss'])
plt.plot(model1.history['val_loss'], label=['val_loss'])
plt.legend()
plt.show()
plt.savefig('loss graph')



In [ ]:
plt.plot(model1.history['accuracy'], label='train-acc')
plt.plot(model1.history['val_accuracy'], label='val-acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')


In [ ]:
# SAVING THE MODEL

from tensorflow.keras.saving import save_model

base_model.save('resnet50_model.h5')


In [ ]:
y_pred = base_model.predict(test_set)

In [ ]:
y_pred

In [36]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
y_pred

In [38]:
from tensorflow.keras.preprocessing import image

In [ ]:
model = load_model('resnet50_model.h5')

In [52]:
img = image.load_img('Test/mercedes/28.jpg', target_size=(224,224))

In [ ]:
x = image.img_to_array(img)
x

In [ ]:
x.shape

In [55]:
x = x/255

In [ ]:
x = np.expand_dims(x, axis=0)
img_data=preprocess_input(x)
img_data.shape

In [ ]:
base_model.predict(img_data)

In [ ]:
a = np.argmax(model.predict(img_data), axis=1)
a